# Model Inference

This notebook details how to infer from the English-Thai machine translation models.

Import library `fairseq`. Not that you need to install from source:

```
git clone https://github.com/pytorch/fairseq
cd fairseq
pip install --editable .
```

If you are using Python 3.6, you might run into [this problem](https://stackoverflow.com/questions/55833509/attributeerror-type-object-callable-has-no-attribute-abc-registry).

Download the `artifacts/` folder from [here](https://www.dropbox.com/sh/za16rr8dp7f2d7h/AACpdb28ZrnmVL7yga0n_Mvaa?dl=0).

In [4]:
from fairseq.models.transformer import TransformerModel
import os, time
import sentencepiece
from mosestokenizer import MosesTokenizer

In [5]:
en_tokenize = MosesTokenizer('en')

I0627 17:23:17.355901 140443652982592 toolwrapper.py:96] executing argv ['perl', '/home/cstorm125/venv/lib/python3.6/site-packages/mosestokenizer/tokenizer-v1.1.perl', '-q', '-l', 'en', '-b', '-a']
I0627 17:23:17.474443 140443652982592 toolwrapper.py:117] spawned process 314


Initialize NMT model


1. Model checkpoint .pt
2. Directory that store vocabulary
2. Directory that SentencePiece model (if the model require)



### th->en

In [3]:
MODEL_BASE_DIR = 'artifacts/th-en/models/'
VOCAB_BASE_DIR = 'artifacts/th-en/vocab/'
BPE_BASE_DIR = 'artifacts/th-en/bpe/'

In [4]:
!ls $MODEL_BASE_DIR

checkpoint_best.pt


In [5]:
!ls $VOCAB_BASE_DIR

dict.en.txt        test.th-en.en.idx  train.th-en.en.idx valid.th-en.en.idx
dict.th.txt        test.th-en.th.bin  train.th-en.th.bin valid.th-en.th.bin
preprocess.log     test.th-en.th.idx  train.th-en.th.idx valid.th-en.th.idx
test.th-en.en.bin  train.th-en.en.bin valid.th-en.en.bin


In [6]:
!ls $BPE_BASE_DIR

spm.en.v-16000.cased.model spm.th.v-16000.cased.model
spm.en.v-16000.cased.vocab spm.th.v-16000.cased.vocab


In [7]:
th2en = TransformerModel.from_pretrained(
    model_name_or_path=MODEL_BASE_DIR,
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path=VOCAB_BASE_DIR,
    bpe='sentencepiece',
    sentencepiece_vocab=os.path.join(BPE_BASE_DIR, 'spm.th.v-16000.cased.model')
)


In [8]:
th2en.translate("""ในวันที่ 25 ธันวาคม ค.ศ. 800 สมเด็จพระสันตะปาปาลีโอที่ 3 ทรงประกอบพิธีราชาภิเษกให้ชาร์เลอมาญ กษัตริย์แห่งชาวแฟรงก์""",
                beam=4)


../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


'On 25 December 800, Pope Leo III gave Charlemagne the Frankish king the coronation.'

In [9]:
th2en.translate('Japan Rail Pass นั้นมีไว้สำหรับผู้ที่มาจากต่างประเทศที่มีความประสงค์เข้าประเทศญี่ปุ่นในฐานะ “ผู้มาเยือนชั่วคราว(Temporary Visitors)” เพื่อการท่องเที่ยว ซึ่งโดยปกติแล้วต้องซื้อ Exchange Order ผ่านทางบริษัทตัวแทนท่องเที่ยวในต่างประเทศ แล้วจึงนำ Exchange Order นั้นมาแลกเป็น Japan Rail Passได้หลังจากที่เข้าประเทศญี่ปุ่นแล้ว')

'Japan Rail Pass is for those from abroad who wish to join Japan as a temporary visitor for tourism purposes. It is usually required to buy Exchange Order via a foreign agent and to trade in Japan as a Japan Rail Pass.'

### en->th

In [10]:
MODEL_BASE_DIR = 'artifacts/en-th/models/'
VOCAB_BASE_DIR = 'artifacts/en-th/vocab/'
BPE_BASE_DIR = 'artifacts/en-th/bpe/'

In [11]:
!ls $MODEL_BASE_DIR

!ls $VOCAB_BASE_DIR

!ls $BPE_BASE_DIR

checkpoint_best.pt
dict.en.txt        test.en-th.en.idx  train.en-th.en.idx valid.en-th.en.idx
dict.th.txt        test.en-th.th.bin  train.en-th.th.bin valid.en-th.th.bin
preprocess.log     test.en-th.th.idx  train.en-th.th.idx valid.en-th.th.idx
test.en-th.en.bin  train.en-th.en.bin valid.en-th.en.bin
spm.en.v-16000.cased.model spm.th.v-16000.cased.model
spm.en.v-16000.cased.vocab spm.th.v-16000.cased.vocab


In [12]:
en2th = TransformerModel.from_pretrained(
    model_name_or_path=MODEL_BASE_DIR,
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path=VOCAB_BASE_DIR,
    bpe='sentencepiece',
    sentencepiece_vocab=os.path.join(BPE_BASE_DIR, 'spm.en.v-16000.cased.model')

)


In [13]:
en2th.translate('Thomas Jefferson (April 13, 1743[a] – July 4, 1826) was an American statesman, diplomat, lawyer, architect, philosopher, and Founding Father who served as the third president of the United States from 1801 to 1809.')

'โทมัส เจฟเฟอร์สัน (Thomas Jefferson: April 13, 1743 [a] – 4 กรกฎาคม ค.ศ. 1826) เป็นบุคคลสัญชาติอเมริกัน นักการทูต นักกฎหมาย สถาปนิก นักปรัชญา และผู้ก่อตั้งพ่อซึ่งทําหน้าที่เป็นประธานาธิบดีคนที่สามของสหรัฐอเมริกาตั้งแต่ ค.ศ. 1801 ถึง ค.ศ. 1809'

In [14]:
en2th.translate('Prolactin is a hormone produced by your pituitary gland which sits at the bottom of the brain. Prolactin causes breasts to grow and develop and causes milk to be made after a baby is born.')

'Prolactin Prolactin เป็นฮอร์โมนที่ผลิตโดยต่อมไร้ท่อซึ่งอยู่ด้านล่างของสมอง prolactin เป็นสาเหตุให้เต้านมเจริญเติบโตและเจริญขึ้นและทําให้เกิดน้ํานมหลังจากทารกเกิด'